# Master video notebook!
This title might be a bit ambitious, but this notebook is supposed to be able to do all of the administration work when it comes to analysing videos. Of course, many functions are already inherent to the kymo_class file and the plot_data file. Ultimately, this notebook is about creating a file hierarchy for the analysis files.

In step one, the Dropbox is scoured for information about videos. If the videos do not have a VideoInfo.txt, the program will look for a .csv, if there is no .csv, the program will look for a .xlsx file. This is currently in conflict with what is happening in the kymo_class.py file, so that one will have to be amended.

### Let's say that there are three ways to initiate a kymograph class:
1. No info file is submitted, and the class will look for such a file itself.
2. An info file is submitted, and the class will use the data in there.
3. A kymograph is submitted, and the class will instantiate with the parameters that are passed with the kymograph.

TODO: Make it so in the kymo_class
TODO: Streamline the variable storage such that edge properties are stored in the edge_analysis class, and video properties are stored in the video_analysis class.

### Below code:
Are just import statements

In [8]:
from IPython.display import clear_output
import re
from amftrack.pipeline.development.high_mag_videos.kymo_class import *
from amftrack.pipeline.development.high_mag_videos.plot_data import (
    save_raw_data,
    plot_summary
)
import sys
import os
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
from tifffile import imwrite
from tqdm import tqdm
%matplotlib widget
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    clean_degree_4,
)
import scipy
import matplotlib as mpl

from amftrack.pipeline.launching.run import (
    run_transfer,
)
from amftrack.pipeline.launching.run_super import run_parallel_transfer

import dropbox
from amftrack.util.dbx import upload_folders, download, read_saved_dropbox_state, save_dropbox_state, load_dbx, download, get_dropbox_folders, get_dropbox_video_folders
from subprocess import call
import logging

logging.basicConfig(stream=sys.stdout, level=logging.debug)
mpl.rcParams['figure.dpi'] = 300


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## File declaration
As this notebook is designed to work with Snellius, two items to separate are the raw video files and the anaylsis. The raw video files are large, bulky and not so easy to flip through. Ideally, the video files would be downloaded and the analysis would be stored on a separate folder structure entirely. That way, large scale analysis of analysis folders can happen when there are thousands of videos in the dataset, without having to have those raw video folders on hand.

### Input and output:
Please give separately the folder where raw video data is stored, and where the analysis will be stored.

In [9]:
videos_folder = "/gpfs/scratch1/shared/amftrackflow/"
analysis_folder = "/gpfs/home6/svstaalduine/Analysis/"

# Module 1: Downloading videos from DropBox

## Dropbox scouring
The below code is meant to scour the dropbox for information files on the videos. It is also to create a list of the videos within a certain database. The expectation at the very least is that one plate contains many videos, all labeled with a number. The code will take this list, and recreate the hierarchy within the Analysis_Output folder.

### Input and output:
The input will be the highest folder of the dropbox that needs to be analyzed. The output will be a DataFrame that can be filtered in the next code block to prepare for downloading.

In [44]:
# dropbox_address = "/DATA/FLUORESCENCE/DATA_NileRed/20230201_Plate558/"
dropbox_address=  "/DATA/MYRISTATE/DATA/2_weeks/20230524_Plate049/"

In [45]:
all_folders_drop, excel_drop, txt_drop = get_dropbox_video_folders(dropbox_address, True)

clear_output(wait=False)

all_folders_drop

,folder,Plate number,Date Imaged,tot_path_drop,video
0,20230524_Plate049_001/Img/,049,20230524,DATA/MYRISTATE/DATA/2_weeks/20230524_Plate049/...,001/Img/
1,20230524_Plate049_002/Img/,049,20230524,DATA/MYRISTATE/DATA/2_weeks/20230524_Plate049/...,002/Img/
2,20230524_Plate049_003/Img/,049,20230524,DATA/MYRISTATE/DATA/2_weeks/20230524_Plate049/...,003/Img/
3,20230524_Plate049_004/Img/,049,20230524,DATA/MYRISTATE/DATA/2_weeks/20230524_Plate049/...,004/Img/
4,20230524_Plate049_005/Img/,049,20230524,DATA/MYRISTATE/DATA/2_weeks/20230524_Plate049/...,005/Img/
5,20230524_Plate049_006/Img/,049,20230524,DATA/MYRISTATE/DATA/2_weeks/20230524_Plate049/...,006/Img/
6,20230524_Plate049_007/Img/,049,20230524,DATA/MYRISTATE/DATA/2_weeks/20230524_Plate049/...,007/Img/
7,20230524_Plate049_008/Img/,049,20230524,DATA/MYRISTATE/DATA/2_weeks/20230524_Plate049/...,008/Img/


In [46]:
excel_addresses = np.array([re.search("^.*Plate.*\/.*Plate.*$", entry, re.IGNORECASE) for entry in excel_drop])
excel_addresses = excel_addresses[excel_addresses != None]
excel_addresses = [address.group(0) for address in excel_addresses]

for address in excel_addresses:
    suffix = address.split('.')[-1]
    print(suffix)

csv


## Dropbox filtering
If you want all videos in a plate or dataset, you can run this block and forget about it.
Otherwise, this block is where filtering can take place to only download videos with certain properties, like imaging mode. Then a file structure will be created in the videos and analysis folder, and videoInfo.txt folders are created for every video, in the analysis folder. The intent here is to have a uniform Analysis folder structure that works with the Morrison setup.

### Input
Use the section between commented lines to filter the DataFrame, otherwise leave blank
### Output
Within the video and analysis folder, a hierarchy will be created to mimic that of the dropbox folder structure, using the filtered DataFrame. Inside the analysis folder hierarchy, the VideoInfo.txt file will be generated. Either from an existing videoInfo.txt, or the excel/csv sheets.

## Downloading
This section, there is one block of code that will ask you one last time whether all of the parameters are correct. The block of code after that will initiate Snellius jobs to download the videos in the DataFrame from the dropbox. Downloading videos is not that costly, but of course we prefer it to be done as efficiently as possible.
### Input:
Nothing
### Output:
Print statement with the DataFrame and the folders where everything will be stored.
Subsequent block of code will download raw video files to the videos folder.

# Module 2: Analysis
Now that the files have been downloaded, it's time to analyse them. In the below code, you'll be able to either do a complete survey of the analysis folder for as many videos as possible, or use the DataFrame of recently downloaded videos to filter for the videos you want to analyse.

### Input:
DataFrame of all videos to be analysed
### Output:
Print statements for all parameters of the analysis session that is about to take place.

## Run Analysis job
Below code will use the aforementioned DataFrame to initiate analysis jobs on Snellius.
## Input:
Snellius job parameters
## Output:
Analysis folder will be populated with analysis tiffs and csv sheets.

# Module 3: Bulk Analysis
## First part: Assemble Edge DataFrame
In this initial part of the bulk analysis, all of the analysis folders will be looked through to find the edge data we're looking for.